In [24]:
import pandas as pd
import numpy as np
import seaborn as sns

## Groupby Aggregations

Performing various aggregations in pandas. 

In [3]:
# Load in sample dataset. This dataset has a good variety of numeric / categorical columns. 
main_df = sns.load_dataset('taxis')
main_df.head(3)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan


Suppose we wish to compute for each `pickup_borough` :
1. The median distance travelled
2. The max distance travelled
3. The total amount paid
4. The total number of trips

### Default `.agg()`

In [4]:
main_df.groupby('pickup_borough').agg({
    'distance' : ['median', 'max'] # For more than one aggregation per measure
    , 'total' : 'sum'
    , 'pickup' : 'count'
})

distance            total pickup
                 median    max       sum  count
pickup_borough                                 
Bronx               4.0  23.61   2253.76     99
Brooklyn            2.6  25.51   7367.48    383
Manhattan           1.5  28.30  87820.23   5268
Queens              6.1  36.70  20800.69    657

`.agg()` provides a quick way to compute aggregate values. However there are limitations. 
1. Multiple aggregations of the same dimension result in the multi column index that isn't as easily interpereted as a conventional column. Can be rectified by the custome renaming as mentioend later on. 
2. Cannot perform custom aggregations. 

### `pd.NamedAgg()`

Calling `.agg()` but this time passing `NamedAgg()` arguments. These allow us to specify : 

- What the resulting aggregated columns are named
- Custom aggregation functions should we choose

__NOTE__ when creating custom aggregation functions, the input will be the pandas Series version of the column specified. 

[Reference Article](https://deanla.com/pandas_named_agg.html)

In [5]:
tdf = main_df.groupby('pickup_borough').agg(
    median_distance = pd.NamedAgg(column='distance', aggfunc='median')
    , max_distance = pd.NamedAgg(column='distance', aggfunc=lambda x : x.max()) # Creating custom agg function
    , total_paid = pd.NamedAgg(column='total', aggfunc='sum')
    , n_trips = pd.NamedAgg(column='total', aggfunc='size')
)
tdf

,median_distance,max_distance,total_paid,n_trips
pickup_borough,,,,
Bronx,4.0,23.61,2253.76,99
Brooklyn,2.6,25.51,7367.48,383
Manhattan,1.5,28.30,87820.23,5268
Queens,6.1,36.70,20800.69,657


The downside to this approach is that it's more verbose and isn't as quick to write. 

## Stacking & Unstacking

Exploration of pivoting / unpivoting using the inbuilt `.stack()` & `.unstack()` functionality. 

[Reference Video](https://www.youtube.com/watch?v=kJsiiPK5sxs)

In [6]:
# Load in sample dataset. This dataset has a good variety of numeric / categorical columns. 
main_df = sns.load_dataset('taxis')
main_df.head(3)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan


Observe that the dataset is in a [tidy format](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html#:~:text=Tidy%20data%20is%20a%20standard,with%20observations%2C%20variables%20and%20types.), where : 

1. Every column is a variable
2. Every row is an observation
3. Every cell is a single value

Data in this format is easily understood.

### Unstacking (or Pivoting) / Pivoting Long to Wide

It is typical to want to aggreagate data by one or more dimensions. Suppose we wanted __the breakdown of the *number of* rides with each payment type by pickup borough__

In [7]:
# Group by the two items
df = main_df.groupby(['payment', 'pickup_borough'])\
    .agg({'pickup' : 'size'})\
    .rename(columns={'pickup' : 'n'})
df

n
payment     pickup_borough      
cash        Bronx             25
            Brooklyn         119
            Manhattan       1397
            Queens           266
credit card Bronx             74
            Brooklyn         261
            Manhattan       3839
            Queens           383

This gives us the multi indexed aggregation. However this view isn't great for making quick comparisons across dimensions (eg. how do cash & credit card payments differ for the Bronx?)

Converting the `pickup_borough` to separate columns, or __pivoting from long to wide__ would solve this problem.

`df.unstack(level=-1)` lets us to this. In short, we turn a specified `level` value (one of the index columns) from an indexing row to a series of columns. 

By default `level=-1` refers to the outermost column index. Note that the __innermost index is to the left, while the outermost index is on the right__

In [8]:
# The boroughs were the second level index, on the right of payment methods
df.unstack(level=-1)

n                          
pickup_borough Bronx Brooklyn Manhattan Queens
payment                                       
cash              25      119      1397    266
credit card       74      261      3839    383

In [9]:
# Here we choose to pivot on the innermost index of payment method instead, so we need to specify the level=0
df = df.unstack(0)
df

n            
payment         cash credit card
pickup_borough                  
Bronx             25          74
Brooklyn         119         261
Manhattan       1397        3839
Queens           266         383

The comparison is now plain to see. However there arises a new issue with the column indexing. 

The columns are now a MultiIndex, with two layers to the columns. 

In [10]:
df.columns

MultiIndex([('n',        'cash'),
            ('n', 'credit card')],
           names=[None, 'payment'])

It is possible to index the data using [tuple indexing](https://pandas.pydata.org/docs/user_guide/advanced.html), however most of the time it easier to simply concat the two layers together. 

In [11]:
df.columns =  ['__'.join(col).strip() for col in df.columns.values]
df

,n__cash,n__credit card
pickup_borough,,
Bronx,25,74
Brooklyn,119,261
Manhattan,1397,3839
Queens,266,383


This manual reset also removes the two layer column names. 

In [12]:
df.columns

Index(['n__cash', 'n__credit card'], dtype='object')

### Stacking (or unpivoting) / Pivoting Wide to Long

This function does the opposite of `unstack()`. We take the pivoted dataframe and convert it __back to its tidy format__.

In [13]:
df.unstack()

                pickup_borough
n__cash         Bronx               25
                Brooklyn           119
                Manhattan         1397
                Queens             266
n__credit card  Bronx               74
                Brooklyn           261
                Manhattan         3839
                Queens             383
dtype: int64

Observe that this causes a problem: the object returned is a series by default. If we want to convert it back to the tidy format we can reset the index.

We will also need to tidy up some of the column names, as well as some column values (for the column that was unpivoted)

In [14]:
df = df.stack().reset_index()\
    .rename(columns = {'level_1' : "payment_method", 0:'n'})

df['payment_method'] = df['payment_method'].str.removeprefix("n__")
df

,pickup_borough,payment_method,n
0,Bronx,cash,25
1,Bronx,credit card,74
2,Brooklyn,cash,119
3,Brooklyn,credit card,261
4,Manhattan,cash,1397
5,Manhattan,credit card,3839
6,Queens,cash,266
7,Queens,credit card,383


Now the dataframe is back to it's original tidy format, although the multi index no longer exists. 

If you wish you may manually set it yourself. 

In [15]:
df.set_index(['payment_method', 'pickup_borough'])

,,n
payment_method,pickup_borough,
cash,Bronx,25
credit card,Bronx,74
cash,Brooklyn,119
credit card,Brooklyn,261
cash,Manhattan,1397
credit card,Manhattan,3839
cash,Queens,266
credit card,Queens,383


Now we are back to the multi-indexed version as a dataframe.

# Transformation

In `SQL` the `OVER()` clause assigns a value for each row in the dataframe. We can do the same thing for pandas dataframes using the `.transform()` function. This is normally done in conjunction with a `groupby()` call first. 

In [16]:
df = main_df.drop(['pickup', 'dropoff'], axis = 1).copy()
df = df[['pickup_borough', 'dropoff_borough', 'dropoff_zone', 'total', 'distance', 'payment', 'passengers']]

Suppose we wanted to <mark>get the mean spending (`total`)</mark> for that record's `dropoff_zone`, saving it as a separate column:

In [17]:
df['tmp1']=df.groupby('dropoff_zone', as_index=False).total.transform(func='mean')

We could also obtain the <mark>difference</mark> between the group mean and record's value:

In [18]:
df['tmp1']=df.groupby('dropoff_zone').total.transform(func = lambda x: x - x.mean())

## Ranking

It is common to perform ranking functions on datasets. In pandas there is a dedicated `rank()` function. This can be called on both groupby objects as well as on the dataframe itself

In [19]:
# Ride total $ rankings within group
df['tmp1']=df.groupby('dropoff_zone').total.rank(method='min')

In [20]:
df

,pickup_borough,dropoff_borough,dropoff_zone,total,distance,payment,passengers,tmp1
0,Manhattan,Manhattan,UN/Turtle Bay South,12.95,1.60,credit card,1,31.0
1,Manhattan,Manhattan,Upper West Side South,9.30,0.79,cash,1,13.0
2,Manhattan,Manhattan,West Village,14.16,1.37,credit card,1,61.0
3,Manhattan,Manhattan,Yorkville West,36.95,7.70,credit card,1,104.0
4,Manhattan,Manhattan,Yorkville West,13.40,2.16,credit card,3,53.0
...,...,...,...,...,...,...,...,...
6428,Manhattan,Manhattan,Central Harlem North,6.36,0.75,credit card,1,11.0
6429,Queens,Bronx,East Concourse/Concourse Village,58.80,18.74,credit card,1,13.0
6430,Brooklyn,Brooklyn,Bushwick North,17.30,4.14,cash,1,5.0
6431,Brooklyn,Brooklyn,East Flatbush/Remsen Village,6.80,1.12,credit card,1,1.0


# Ranking & Picking

When aggregating data across various sources, being able to select entries based on specific criteria (e.g ones with more recent or complete information) is advantageous

In [37]:
df = pd.DataFrame(
    {
        'Class': ['B', 'A', 'B', 'B', 'A', 'A']
        , 'Trustworthiness':["Low", "Low", "High", "High", "High", np.NaN]
        , "Rating Date": ['2024-01-09', '2024-02-03', '2024-01-16', '2024-01-14', '2024-01-01', '2023-12-30']
        , 'Score': [50, 98, 90, 100, 94, 96]
    }
)
df['Rating Date'] = pd.to_datetime(df['Rating Date'])
df

,Class,Trustworthiness,Rating Date,Score
0,B,Low,2024-01-09,50
1,A,Low,2024-02-03,98
2,B,High,2024-01-16,90
3,B,High,2024-01-14,100
4,A,High,2024-01-01,94
5,A,NaN,2023-12-30,96


Suppose for each `class`, we wanted to take the most recent score with highest `Trustworthiness`. 

It is more important to us that the record with the greater `Trustworthiness` be chosen, even if it was rated later. 

The first thing to do is to ensure all our sorting fields have an order to them. If this is not inherent (as in a numeric or date type field) then we can impose this order using the `Categorical` datatype:

In [38]:
df['Trustworthiness'] = pd.Categorical(
    df['Trustworthiness']
    # Thses are in "Ascending" order, which is the default for the pandas sort_values() call
    , categories=['High', 'Low']
    , ordered=True
)

In [39]:
df.sort_values(['Trustworthiness', 'Rating Date'], ascending=[True, False])

,Class,Trustworthiness,Rating Date,Score
2,B,High,2024-01-16,90
3,B,High,2024-01-14,100
4,A,High,2024-01-01,94
1,A,Low,2024-02-03,98
0,B,Low,2024-01-09,50
5,A,NaN,2023-12-30,96


Observe that the sroting is stable. Now we simply need to pick the first two records that appear based on our desired grouping, in this case the `Class`

In [40]:
df.sort_values(['Trustworthiness', 'Rating Date'], ascending=[True, False]).groupby('Class', as_index=False).first()

,Class,Trustworthiness,Rating Date,Score
0,A,High,2024-01-01,94
1,B,High,2024-01-16,90


# Forward / Backward filling

When aggregating data we often come across missing information. In such scenarios it may be helpful to fill in missing fields with information from other records.

In [42]:
df = pd.DataFrame(
    {
        'Class': ['B', 'A', 'B', 'B', 'A', 'A']
        , 'Trustworthiness':["Low", "Low", np.NaN, "High", "High", np.NaN]
        , "Rating Date": ['2024-01-09', np.NaN, '2024-01-16', '2024-01-14', np.NaN, '2023-12-30']
        , 'Score': [50, 98, np.NaN, 100, 94, np.NaN]
    }
)
df['Rating Date'] = pd.to_datetime(df['Rating Date'])
df

,Class,Trustworthiness,Rating Date,Score
0,B,Low,2024-01-09,50.0
1,A,Low,NaT,98.0
2,B,NaN,2024-01-16,NaN
3,B,High,2024-01-14,100.0
4,A,High,NaT,94.0
5,A,NaN,2023-12-30,NaN


We could backward / forward fill the entries on the dataframe level:

In [43]:
df.ffill()

,Class,Trustworthiness,Rating Date,Score
0,B,Low,2024-01-09,50.0
1,A,Low,2024-01-09,98.0
2,B,Low,2024-01-16,98.0
3,B,High,2024-01-14,100.0
4,A,High,2024-01-14,94.0
5,A,High,2023-12-30,94.0


Or we could do the filling in based on a specific group instead. For example we could only fill in missing fields using other records from the same `Class`

In [44]:
# First set_index to avoid having to replace the requisite columns in the main dataframe
df.set_index('Class').groupby('Class').ffill().bfill().reset_index()

,Class,Trustworthiness,Rating Date,Score
0,B,Low,2024-01-09,50.0
1,A,Low,2024-01-16,98.0
2,B,Low,2024-01-16,50.0
3,B,High,2024-01-14,100.0
4,A,High,2023-12-30,94.0
5,A,High,2023-12-30,94.0


Notice that index 2 now has a score of 50 but when the filling was done on the dataframe level it took from the last entry of a different `Class` and had a score of 98 filled in. 